In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.font_manager as fm
import matplotlib as mpl
from collections import defaultdict
import ast

In [6]:
plt.rcParams['font.family'] = 'AppleGothic'
mpl.rcParams['axes.unicode_minus'] = False

In [7]:
def safe_literal_eval(val):
    """
    주어진 값이 문자열로 되어 있고 '['로 시작하면 ast.literal_eval을 시도하고,
    그렇지 않으면 그대로 반환하는 함수.
    """
    if isinstance(val, str) and val.strip().startswith('['):
        try:
            return ast.literal_eval(val)
        except Exception as e:
            print("literal_eval 오류:", e, "값:", val)
            return None
    # 문자열이 아니면 그대로 반환 (예: 이미 리스트 또는 numpy 배열인 경우)
    return val

In [8]:
# 1. 병합된 데이터 로드 (CSV 파일)
tsf_file_path = "./dataset/final_dataset/M4_Quarterly_Merged.csv"  # CSV 파일 경로
df_q = pd.read_csv(tsf_file_path)

# 2. 병합된 데이터에서 'category' 컬럼을 기준으로 도메인별 그룹화
domain_time_series = defaultdict(dict)
for idx, row in df_q.iterrows():
    category = row['category']  # 도메인 정보 (예: Macro, Finance 등)
    ts_id = row['series_name']
    # series_value 컬럼은 문자열 형태의 리스트이므로 안전하게 변환
    values = safe_literal_eval(row['series_value'])
    if values is None:
        continue
    domain_time_series[category][ts_id] = (row['start_timestamp'], values)

In [9]:
domain_time_series

In [10]:
def check_missing_values(domain_time_series):
    """
    시계열 데이터에서 결측값(NaN)을 확인합니다.
    """
    missing_stats = {
        'total_series': 0,
        'series_with_missing': 0,
        'total_values': 0,
        'total_missing': 0,
        'series_details': []
    }
    
    # 먼저 모든 시계열의 총 수를 계산
    total_series = 0
    for domain, series_dict in domain_time_series.items():
        total_series += len(series_dict)
    
    missing_stats['total_series'] = total_series
    
    # 각 도메인과 시리즈를 반복
    for domain, series_dict in domain_time_series.items():
        for ts_id, (start_time, values) in series_dict.items():
            total = len(values)
            missing = sum(np.isnan(v) for v in values)
            missing_ratio = missing / total if total > 0 else 0
            
            missing_stats['total_values'] += total
            missing_stats['total_missing'] += missing
            
            if missing > 0:
                missing_stats['series_with_missing'] += 1
                missing_stats['series_details'].append({
                    'domain': domain,
                    'ts_id': ts_id,
                    'total_values': total,
                    'missing_values': missing,
                    'missing_ratio': missing_ratio
                })
    
    # 전체 결측치 비율 계산
    if missing_stats['total_values'] > 0:
        missing_stats['overall_missing_ratio'] = missing_stats['total_missing'] / missing_stats['total_values']
    else:
        missing_stats['overall_missing_ratio'] = 0
        
    return missing_stats

# 결측값 확인
missing_stats = check_missing_values(domain_time_series)
print("\n===== 결측값 통계 =====")
print(f"전체 시계열 수: {missing_stats['total_series']}")
print(f"결측값이 있는 시계열 수: {missing_stats['series_with_missing']}")
print(f"전체 데이터 포인트 수: {missing_stats['total_values']}")
print(f"전체 결측값 수: {missing_stats['total_missing']}")
print(f"전체 결측치 비율: {missing_stats['overall_missing_ratio']:.6f} ({missing_stats['overall_missing_ratio']*100:.4f}%)")

# 결측값이 있는 경우 세부 정보 표시
if missing_stats['series_with_missing'] > 0:
    print("\n결측값이 있는 시계열 세부 정보:")
    for detail in missing_stats['series_details']:
        print(f" {detail['domain']}-{detail['ts_id']}: {detail['missing_values']} 결측값 / {detail['total_values']} 전체 ({detail['missing_ratio']*100:.2f}%)")

In [11]:
def check_time_series_lengths(domain_time_series):
    """
    시계열 데이터의 길이 분포를 확인하고 요약합니다.
    """
    # 각 시계열의 길이 저장
    lengths = {}
    total_series = 0
    
    # 도메인과 시계열 순회
    for domain, series_dict in domain_time_series.items():
        for ts_id, (start_time, values) in series_dict.items():
            key = f"{domain}-{ts_id}"  # 도메인과 시계열 ID를 합쳐서 고유 키 생성
            lengths[key] = len(values)
            total_series += 1
    
    # 길이 분포 요약
    length_counts = {}
    for length in set(lengths.values()):
        length_counts[length] = list(lengths.values()).count(length)
    
    # 정렬된 길이 분포
    sorted_length_counts = dict(sorted(length_counts.items()))
    
    # 요약 통계
    min_length = min(lengths.values()) if lengths else 0
    max_length = max(lengths.values()) if lengths else 0
    avg_length = sum(lengths.values()) / len(lengths) if lengths else 0
    
    print(f"시계열 총 개수: {len(lengths)}")
    print(f"최소 길이: {min_length}")
    print(f"최대 길이: {max_length}")
    print(f"평균 길이: {avg_length:.2f}")
    
    print("\n길이 분포:")
    for length, count in sorted_length_counts.items():
        print(f" 길이 {length}: {count}개 시계열 ({count/len(lengths)*100:.2f}%)")
    
    # 가장 짧은 시계열과 가장 긴 시계열 찾기
    if lengths:
        shortest_ts = min(lengths, key=lengths.get)
        longest_ts = max(lengths, key=lengths.get)
        print(f"\n가장 짧은 시계열: {shortest_ts} (길이: {lengths[shortest_ts]})")
        print(f"가장 긴 시계열: {longest_ts} (길이: {lengths[longest_ts]})")
    
    # 시작 시간 분포 확인
    start_times = {}
    for domain, series_dict in domain_time_series.items():
        for ts_id, (start_time, _) in series_dict.items():
            if isinstance(start_time, str):
                start_time_str = start_time
            else:
                # datetime 객체라면 문자열로 변환
                try:
                    start_time_str = start_time.strftime('%Y-%m-%d')
                except:
                    start_time_str = str(start_time)
                    
            if start_time_str not in start_times:
                start_times[start_time_str] = 0
            start_times[start_time_str] += 1
    
    print("\n시작 시간 분포:")
    for start_time, count in sorted(start_times.items()):
        print(f" {start_time}: {count}개 시계열")
    
    return lengths

# 함수 실행
lengths = check_time_series_lengths(domain_time_series)

# 시각화
import matplotlib.pyplot as plt

# 시계열 길이 분포 히스토그램
plt.figure(figsize=(12, 6))
plt.hist(list(lengths.values()), bins=20)
plt.title('시계열 길이 분포')
plt.xlabel('길이')
plt.ylabel('시계열 개수')
plt.grid(True)
plt.show()

# 수정된 산점도 - 도메인별로 색상 구분
plt.figure(figsize=(14, 7))

# 도메인 목록 추출 (키에서 하이픈 이전 부분)
domains = list(set([key.split('-')[0] for key in lengths.keys()]))
domain_colors = plt.cm.tab10(np.linspace(0, 1, len(domains)))

# 도메인별로 다른 색상 사용
for i, domain in enumerate(domains):
    domain_keys = [k for k in lengths.keys() if k.startswith(domain+'-')]
    domain_values = [lengths[k] for k in domain_keys]
    
    # 시리즈 ID 부분만 추출 (하이픈 이후 부분)
    series_ids = [k.split('-')[1] for k in domain_keys]
    
    plt.scatter(range(len(domain_keys)), domain_values, 
                color=domain_colors[i], alpha=0.7, label=domain)

plt.title('도메인 및 시계열 ID별 길이')
plt.xlabel('시계열 인덱스')
plt.ylabel('길이')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 도메인별 길이 박스플롯
plt.figure(figsize=(14, 7))
domain_data = []
domain_names = []

for domain in domains:
    domain_keys = [k for k in lengths.keys() if k.startswith(domain+'-')]
    domain_data.append([lengths[k] for k in domain_keys])
    domain_names.append(f"{domain} (n={len(domain_keys)})")

plt.boxplot(domain_data, labels=domain_names)
plt.title('도메인별 시계열 길이 분포')
plt.ylabel('길이')
plt.grid(True, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [12]:
def calculate_basic_statistics(domain_time_series):
    """
    각 시계열의 기본 통계량을 계산합니다.
    """
    stats = []
    
    # 각 도메인과 시계열 반복
    for domain, series_dict in domain_time_series.items():
        for ts_id, (start_time, values) in series_dict.items():
            # NaN 값 제외한 유효한 값들만 사용
            values_clean = np.array([v for v in values if not np.isnan(v)])
            
            if len(values_clean) > 0:
                # 기본 통계량 계산
                stat = {
                    'domain': domain,
                    'ts_id': ts_id,
                    'length': len(values),
                    'mean': np.mean(values_clean),
                    'median': np.median(values_clean),
                    'std': np.std(values_clean),
                    'min': np.min(values_clean),
                    'max': np.max(values_clean),
                    'range': np.max(values_clean) - np.min(values_clean),
                    'cv': np.std(values_clean) / np.mean(values_clean) if np.mean(values_clean) != 0 else np.nan
                }
                stats.append(stat)
    
    # DataFrame으로 변환
    stats_df = pd.DataFrame(stats)
    return stats_df

# 기본 통계량 계산 및 확인
statistics_df = calculate_basic_statistics(domain_time_series)

# 전체 통계량 요약
print("전체 시계열 통계량 요약:")
print(statistics_df.describe())

# 도메인별 통계량 요약
print("\n도메인별 통계량 요약:")
for domain in statistics_df['domain'].unique():
    domain_stats = statistics_df[statistics_df['domain'] == domain]
    print(f"\n== {domain} 도메인 통계량 (총 {len(domain_stats)}개 시계열) ==")
    print(domain_stats.describe())
    
# 도메인별 통계량 시각화
import matplotlib.pyplot as plt
import seaborn as sns

# 도메인별 평균값 비교
plt.figure(figsize=(14, 8))
sns.boxplot(x='domain', y='mean', data=statistics_df)
plt.title('도메인별 시계열 평균값 분포')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

# 도메인별 표준편차 비교
plt.figure(figsize=(14, 8))
sns.boxplot(x='domain', y='std', data=statistics_df)
plt.title('도메인별 시계열 표준편차 분포')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

# 도메인별 변동계수(CV) 비교
plt.figure(figsize=(14, 8))
sns.boxplot(x='domain', y='cv', data=statistics_df)
plt.title('도메인별 시계열 변동계수(CV) 분포')
plt.xticks(rotation=45)
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

# 평균 vs 표준편차 산점도 (도메인별 색상)
plt.figure(figsize=(14, 8))
domains = statistics_df['domain'].unique()
for i, domain in enumerate(domains):
    domain_data = statistics_df[statistics_df['domain'] == domain]
    plt.scatter(domain_data['mean'], domain_data['std'], 
                alpha=0.7, label=domain)

plt.title('시계열 평균 vs 표준편차 (도메인별)')
plt.xlabel('평균')
plt.ylabel('표준편차')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 도메인별 주요 통계량 요약 테이블
domain_summary = statistics_df.groupby('domain').agg({
    'length': ['count', 'mean'],
    'mean': ['mean', 'min', 'max'],
    'median': ['mean', 'min', 'max'],
    'std': ['mean', 'min', 'max'],
    'range': ['mean', 'min', 'max'],
    'cv': ['mean', 'min', 'max']
})

print("\n도메인별 통계량 요약 테이블:")
print(domain_summary)

In [10]:
def apply_log_transformation(domain_time_series):
    """
    시계열 데이터에 로그 변환을 적용하고 변환 전/후 통계를 비교합니다.
    각 도메인별 샘플 시계열에 대한 변환 전/후를 시각화합니다.
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    from scipy import stats
    import random
    from matplotlib.gridspec import GridSpec
    
    # 원본 데이터 통계량 계산
    original_stats = []
    
    # 로그 변환 데이터 저장용 구조
    log_transformed_series = {}
    
    # 각 도메인별 샘플 시계열 ID 저장 (랜덤 선택)
    sample_series_ids = {}
    
    # 각 도메인과 시계열에 대한 처리
    for domain, series_dict in domain_time_series.items():
        if domain not in log_transformed_series:
            log_transformed_series[domain] = {}
            
        # 이 도메인의 랜덤 시계열 ID 선택 (변환 전/후 비교용)
        sample_series_ids[domain] = random.choice(list(series_dict.keys()))
            
        for ts_id, (start_time, values) in series_dict.items():
            # 원본 데이터 통계 계산
            values_clean = np.array([v for v in values if not np.isnan(v)])
            
            if len(values_clean) > 0:
                stat = {
                    'domain': domain,
                    'ts_id': ts_id,
                    'length': len(values),
                    'mean': np.mean(values_clean),
                    'median': np.median(values_clean),
                    'std': np.std(values_clean),
                    'min': np.min(values_clean),
                    'max': np.max(values_clean),
                    'range': np.max(values_clean) - np.min(values_clean),
                    'cv': np.std(values_clean) / np.mean(values_clean) if np.mean(values_clean) != 0 else np.nan,
                    'skewness': stats.skew(values_clean) if len(values_clean) > 2 else np.nan,
                    'transformation': 'original'
                }
                original_stats.append(stat)
                
                # 로그 변환 적용 (NaN 값은 그대로 유지)
                log_values = np.array([np.log(v) if not np.isnan(v) else np.nan for v in values])
                log_transformed_series[domain][ts_id] = (start_time, log_values)
    
    # 로그 변환 데이터 통계량 계산
    log_stats = []
    
    for domain, series_dict in log_transformed_series.items():
        for ts_id, (start_time, values) in series_dict.items():
            # 로그 변환 데이터 통계 계산
            values_clean = np.array([v for v in values if not np.isnan(v)])
            
            if len(values_clean) > 0:
                stat = {
                    'domain': domain,
                    'ts_id': ts_id,
                    'length': len(values),
                    'mean': np.mean(values_clean),
                    'median': np.median(values_clean),
                    'std': np.std(values_clean),
                    'min': np.min(values_clean),
                    'max': np.max(values_clean),
                    'range': np.max(values_clean) - np.min(values_clean),
                    'cv': np.std(values_clean) / np.mean(values_clean) if np.mean(values_clean) != 0 else np.nan,
                    'skewness': stats.skew(values_clean) if len(values_clean) > 2 else np.nan,
                    'transformation': 'log'
                }
                log_stats.append(stat)
    
    # 통계량 DataFrame 생성
    original_df = pd.DataFrame(original_stats)
    log_df = pd.DataFrame(log_stats)
    
    # 두 데이터프레임 합치기
    combined_df = pd.concat([original_df, log_df])
    
    # 도메인별 로그 변환 전후 비교
    domains = original_df['domain'].unique()
    
    # 각 도메인별로 로그 변환 전후 비교 시각화
    for domain in domains:
        domain_original = original_df[original_df['domain'] == domain]
        domain_log = log_df[log_df['domain'] == domain]
        
        # 기본 통계량 출력
        print(f"\n=== {domain} 도메인 로그 변환 전후 비교 ===")
        
        print("\n원본 데이터 통계:")
        print(domain_original[['mean', 'median', 'std', 'min', 'max', 'cv', 'skewness']].describe())
        
        print("\n로그 변환 데이터 통계:")
        print(domain_log[['mean', 'median', 'std', 'min', 'max', 'cv', 'skewness']].describe())
        
        # 원본과 로그 변환 히스토그램 비교 (평균값 분포만 남김)
        plt.figure(figsize=(15, 6))
        
        plt.subplot(1, 2, 1)
        sns.histplot(domain_original['mean'], kde=True)
        plt.title(f'{domain} 도메인 - 원본 데이터 평균 분포')
        plt.xlabel('값')
        plt.ylabel('빈도')
        
        plt.subplot(1, 2, 2)
        sns.histplot(domain_log['mean'], kde=True)
        plt.title(f'{domain} 도메인 - 로그 변환 데이터 평균 분포')
        plt.xlabel('값 (로그 스케일)')
        plt.ylabel('빈도')
        
        plt.tight_layout()
        plt.show()
        
        # 샘플 시계열 데이터 변환 전/후 비교
        sample_id = sample_series_ids[domain]
        
        # 원본 시계열 데이터
        orig_start_time, orig_values = domain_time_series[domain][sample_id]
        # 로그 변환된 시계열 데이터
        log_start_time, log_values = log_transformed_series[domain][sample_id]
        
        # NaN이 아닌 값만 사용하여 시각화
        valid_indices = ~np.isnan(orig_values)
        valid_orig_values = np.array(orig_values)[valid_indices]
        valid_log_values = np.array(log_values)[valid_indices]
        time_indices = np.arange(len(valid_orig_values))
        
        # 변환 전후 차이를 더 명확하게 보여주는 시각화
        fig = plt.figure(figsize=(15, 12))
        gs = GridSpec(3, 2, figure=fig)
        
        # 원본 데이터
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.plot(time_indices, valid_orig_values, 'b-')
        ax1.set_title(f'{domain} 도메인 샘플 시계열 (ID: {sample_id}) - 원본 데이터')
        ax1.set_ylabel('값')
        ax1.grid(True)
        
        # 로그 변환 데이터
        ax2 = fig.add_subplot(gs[0, 1])
        ax2.plot(time_indices, valid_log_values, 'r-')
        ax2.set_title(f'{domain} 도메인 샘플 시계열 (ID: {sample_id}) - 로그 변환 데이터')
        ax2.set_ylabel('로그 값')
        ax2.grid(True)
        
        # 쌍축 그래프 (같은 그래프에 두 데이터 표시)
        ax3 = fig.add_subplot(gs[1, :])
        line1 = ax3.plot(time_indices, valid_orig_values, 'b-', label='원본 데이터')
        ax3.set_ylabel('원본 값', color='b')
        ax3.tick_params(axis='y', labelcolor='b')
        ax3.grid(True)
        
        ax3_twin = ax3.twinx()
        line2 = ax3_twin.plot(time_indices, valid_log_values, 'r-', label='로그 변환 데이터')
        ax3_twin.set_ylabel('로그 값', color='r')
        ax3_twin.tick_params(axis='y', labelcolor='r')
        
        # 범례 추가
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        ax3.legend(lines, labels, loc='upper right')
        ax3.set_title('원본 vs 로그 변환 (쌍축 - 스케일 차이 확인)')
        
        # 데이터 분포 비교
        ax4 = fig.add_subplot(gs[2, 0])
        ax4.hist(valid_orig_values, bins=20, alpha=0.7, color='b')
        ax4.set_title('원본 데이터 분포')
        ax4.set_xlabel('값')
        ax4.set_ylabel('빈도')
        ax4.grid(True)
        
        ax5 = fig.add_subplot(gs[2, 1])
        ax5.hist(valid_log_values, bins=20, alpha=0.7, color='r')
        ax5.set_title('로그 변환 데이터 분포')
        ax5.set_xlabel('로그 값')
        ax5.set_ylabel('빈도')
        ax5.grid(True)
        
        plt.tight_layout()
        plt.show()
        
        # 변동성 패턴 비교 (이동 표준편차)
        if len(valid_orig_values) > 10:  # 충분한 데이터가 있는 경우에만
            window_size = max(5, len(valid_orig_values) // 10)  # 적절한 윈도우 크기 설정
            
            # 이동 표준편차 계산
            rolling_std_orig = []
            rolling_std_log = []
            
            for i in range(len(valid_orig_values) - window_size + 1):
                rolling_std_orig.append(np.std(valid_orig_values[i:i+window_size]))
                rolling_std_log.append(np.std(valid_log_values[i:i+window_size]))
                
            # 변동성 정규화 (비교를 위해)
            if np.mean(rolling_std_orig) > 0 and np.mean(rolling_std_log) > 0:
                norm_std_orig = np.array(rolling_std_orig) / np.mean(rolling_std_orig)
                norm_std_log = np.array(rolling_std_log) / np.mean(rolling_std_log)
                
                plt.figure(figsize=(15, 6))
                plt.plot(range(len(norm_std_orig)), norm_std_orig, 'b-', 
                         label='원본 데이터 정규화된 이동 표준편차')
                plt.plot(range(len(norm_std_log)), norm_std_log, 'r-', 
                         label='로그 변환 데이터 정규화된 이동 표준편차')
                plt.title('시간에 따른 변동성 비교 (정규화된 이동 표준편차)')
                plt.xlabel('시간 인덱스')
                plt.ylabel('정규화된 표준편차')
                plt.legend()
                plt.grid(True)
                plt.show()
    
    # 전체 도메인 변환 전후 왜도 비교
    plt.figure(figsize=(14, 8))
    
    skew_comparison = pd.DataFrame({
        'domain': domains,
        'original_skew': [original_df[original_df['domain'] == d]['skewness'].mean() for d in domains],
        'log_skew': [log_df[log_df['domain'] == d]['skewness'].mean() for d in domains]
    })
    
    bar_width = 0.35
    x = np.arange(len(domains))
    
    plt.bar(x - bar_width/2, skew_comparison['original_skew'], bar_width, label='원본 데이터')
    plt.bar(x + bar_width/2, skew_comparison['log_skew'], bar_width, label='로그 변환 데이터')
    
    plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    plt.xticks(x, domains)
    plt.title('도메인별 평균 왜도 비교 (변환 전후)')
    plt.xlabel('도메인')
    plt.ylabel('평균 왜도')
    plt.legend()
    plt.grid(True, axis='y')
    plt.show()
    
    return log_transformed_series, combined_df

# 함수 실행
log_transformed_series, transformation_stats = apply_log_transformation(domain_time_series)

In [11]:
log_transformed_series

In [12]:
def apply_and_visualize_differencing(domain_time_series, domain, ts_id=None):
    """
    특정 도메인의 시계열 데이터에 다양한 차분을 적용하고 시각화합니다:
    1. 1차 차분
    2. 2차 차분
    3. 계절 차분(4 주기)
    4. 계절 차분 + 1차 차분
    5. 계절 차분 + 2차 차분
    
    각 단계별로 ADF 및 KPSS 정상성 검정을 수행합니다.
    """
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import random
    from statsmodels.tsa.stattools import adfuller, kpss
    from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
    import warnings
    
    # 경고 숨기기
    warnings.filterwarnings("ignore")
    
    # 도메인에서 시계열 선택
    if ts_id is None:
        ts_id = random.choice(list(domain_time_series[domain].keys()))
    
    start_time, values = domain_time_series[domain][ts_id]
    values_np = np.array(values)
    
    # 결측값 처리
    mask = ~np.isnan(values_np)
    values_clean = values_np[mask]
    
    # 값이 너무 적으면 다른 시계열 선택
    if len(values_clean) < 10:
        print(f"시계열 {ts_id}의 유효 데이터가 너무 적습니다. 다른 시계열을 선택하세요.")
        return None
    
    # pandas Series로 변환 (날짜 인덱스 포함)
    if isinstance(start_time, str):
        start_time = pd.to_datetime(start_time)
    
    # 인덱스가 중요하지 않다면 정수 인덱스로 대체 가능
    original_series = pd.Series(values_clean)
    
    # 차분 적용
    # 1. 1차 차분
    diff1 = original_series.diff().dropna()
    
    # 2. 2차 차분
    diff2 = diff1.diff().dropna()
    
    # 3. 계절 차분 (4 주기)
    if len(original_series) > 4:  # 계절 차분을 위한 최소 길이 확인
        seasonal_diff = original_series.diff(periods=4).dropna()
    else:
        seasonal_diff = pd.Series([])  # 빈 시리즈 생성
    
    # 4. 계절 차분 + 1차 차분
    if len(seasonal_diff) > 1:
        seasonal_diff1 = seasonal_diff.diff().dropna()
    else:
        seasonal_diff1 = pd.Series([])
    
    # 5. 계절 차분 + 2차 차분
    if len(seasonal_diff1) > 1:
        seasonal_diff2 = seasonal_diff1.diff().dropna()
    else:
        seasonal_diff2 = pd.Series([])
    
    # 차분 적용 결과 시각화
    fig, axes = plt.subplots(6, 1, figsize=(14, 18), sharex=True)
    
    # 원본 시계열
    original_series.plot(ax=axes[0], title=f'원본 시계열 ({domain} 도메인, {ts_id})')
    axes[0].set_ylabel('값')
    axes[0].grid(True)
    
    # 1차 차분
    if len(diff1) > 0:
        diff1.plot(ax=axes[1], title=f'1차 차분')
    else:
        axes[1].text(0.5, 0.5, "데이터 부족으로 계산 불가", ha='center', va='center')
    axes[1].set_ylabel('1차 차분 값')
    axes[1].grid(True)
    
    # 2차 차분
    if len(diff2) > 0:
        diff2.plot(ax=axes[2], title=f'2차 차분')
    else:
        axes[2].text(0.5, 0.5, "데이터 부족으로 계산 불가", ha='center', va='center')
    axes[2].set_ylabel('2차 차분 값')
    axes[2].grid(True)
    
    # 계절 차분 (4 주기)
    if len(seasonal_diff) > 0:
        seasonal_diff.plot(ax=axes[3], title=f'계절 차분 (4 주기)')
    else:
        axes[3].text(0.5, 0.5, "데이터 부족으로 계산 불가", ha='center', va='center')
    axes[3].set_ylabel('계절 차분 값')
    axes[3].grid(True)
    
    # 계절 차분 + 1차 차분
    if len(seasonal_diff1) > 0:
        seasonal_diff1.plot(ax=axes[4], title=f'계절 차분 + 1차 차분')
    else:
        axes[4].text(0.5, 0.5, "데이터 부족으로 계산 불가", ha='center', va='center')
    axes[4].set_ylabel('계절 + 1차 차분 값')
    axes[4].grid(True)
    
    # 계절 차분 + 2차 차분
    if len(seasonal_diff2) > 0:
        seasonal_diff2.plot(ax=axes[5], title=f'계절 차분 + 2차 차분')
    else:
        axes[5].text(0.5, 0.5, "데이터 부족으로 계산 불가", ha='center', va='center')
    axes[5].set_ylabel('계절 + 2차 차분 값')
    axes[5].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # 정상성 검정 함수
    def run_stationarity_tests(series, name):
        print(f"\n=== {name} 정상성 검정 ===")
        
        if len(series) < 8:  # 테스트에 필요한 최소 데이터 길이
            print(f"데이터 길이가 충분하지 않아 검정을 수행할 수 없습니다.")
            return
        
        try:
            # ADF 검정
            adf_result = adfuller(series.dropna())
            print(f"ADF 검정 결과:")
            print(f'  ADF 통계량: {adf_result[0]:.4f}')
            print(f'  p-value: {adf_result[1]:.4f}')
            print(f'  정상성 여부: {"정상" if adf_result[1] < 0.05 else "비정상"}')
        except:
            print("ADF 검정 실패 - 시계열 특성이나 길이 문제일 수 있습니다.")
        
        try:
            # KPSS 검정
            kpss_result = kpss(series.dropna())
            print(f"KPSS 검정 결과:")
            print(f'  KPSS 통계량: {kpss_result[0]:.4f}')
            print(f'  p-value: {kpss_result[1]:.4f}')
            print(f'  정상성 여부: {"정상" if kpss_result[1] > 0.05 else "비정상"}')
        except:
            print("KPSS 검정 실패 - 시계열 특성이나 길이 문제일 수 있습니다.")
    
    # 각 시계열에 대한 정상성 검정
    run_stationarity_tests(original_series, "원본 시계열")
    run_stationarity_tests(diff1, "1차 차분")
    run_stationarity_tests(diff2, "2차 차분")
    run_stationarity_tests(seasonal_diff, "계절 차분")
    run_stationarity_tests(seasonal_diff1, "계절 차분 + 1차 차분")
    run_stationarity_tests(seasonal_diff2, "계절 차분 + 2차 차분")
    
    # 각 시계열의 ACF/PACF 시각화
    fig, axes = plt.subplots(6, 2, figsize=(15, 24))
    
    # 차분별 시계열 및 이름 목록
    series_list = [
        (original_series, "원본 시계열"),
        (diff1, "1차 차분"),
        (diff2, "2차 차분"),
        (seasonal_diff, "계절 차분"),
        (seasonal_diff1, "계절 차분 + 1차 차분"),
        (seasonal_diff2, "계절 차분 + 2차 차분")
    ]
    
    # 각 시계열에 대한 ACF/PACF 시각화
    for i, (series, name) in enumerate(series_list):
        if len(series) > 3:  # ACF/PACF에 필요한 최소 데이터 길이 확인
            try:
                # ACF 플롯
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    plot_acf(series.dropna(), lags=min(48, len(series) // 2), ax=axes[i, 0])
                axes[i, 0].set_title(f'{name} ACF')
                
                # PACF 플롯
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    plot_pacf(series.dropna(), lags=min(48, len(series) // 2), ax=axes[i, 1])
                axes[i, 1].set_title(f'{name} PACF')
            except:
                axes[i, 0].text(0.5, 0.5, f"ACF 계산 실패", ha='center', va='center')
                axes[i, 1].text(0.5, 0.5, f"PACF 계산 실패", ha='center', va='center')
        else:
            axes[i, 0].text(0.5, 0.5, f"데이터 부족으로 계산 불가", ha='center', va='center')
            axes[i, 1].text(0.5, 0.5, f"데이터 부족으로 계산 불가", ha='center', va='center')
        
        axes[i, 0].set_title(f'{name} ACF')
        axes[i, 1].set_title(f'{name} PACF')
    
    plt.tight_layout()
    plt.show()
    
    # 결과 시계열 반환
    return {
        "domain": domain,
        "ts_id": ts_id,
        "original": original_series,
        "diff1": diff1,
        "diff2": diff2,
        "seasonal_diff": seasonal_diff,
        "seasonal_diff1": seasonal_diff1,
        "seasonal_diff2": seasonal_diff2
    }

# 각 도메인별로 시계열 하나씩 선택하여 차분 분석 실행
def analyze_domain_differencing(domain_time_series):
    """
    각 도메인별로 시계열 하나씩 선택하여 차분 분석을 수행합니다.
    """
    import random
    
    differencing_results = {}
    
    for domain in domain_time_series.keys():
        print(f"\n{'='*50}")
        print(f"도메인: {domain} 차분 분석")
        print(f"{'='*50}")
        
        # 각 도메인에서 데이터 길이가 충분한 시계열 찾기
        valid_ts_ids = []
        for ts_id, (_, values) in domain_time_series[domain].items():
            if sum(~np.isnan(values)) >= 20:  # 최소 20개의 유효한 데이터 포인트 필요
                valid_ts_ids.append(ts_id)
        
        if not valid_ts_ids:
            print(f"{domain} 도메인에 충분한 데이터를 가진 시계열이 없습니다.")
            continue
        
        # 충분한 데이터를 가진 시계열 중 랜덤하게 선택
        ts_id = random.choice(valid_ts_ids)
        
        # 선택된 시계열에 대해 차분 분석 수행
        result = apply_and_visualize_differencing(domain_time_series, domain, ts_id)
        if result:
            differencing_results[domain] = result
        
    return differencing_results

# 모든 도메인에 대해 차분 분석 실행
all_domain_differencing = analyze_domain_differencing(log_transformed_series)

### 수동 파라미터 선정

MACRO: SARIMA(2,1,1)(0,1,0)4  
MICRO: SARIMA(2,1,1)(0,1,0)4  
DEMOGRAPHIC: SARIMA(0,1,0)(0,0,0)4  
INDUSTRY: SARIMA(0,1,0)(0,1,2)4  
FINANCE: SARIMA(2,2,1)(0,0,0)4  
OTHER: SARIMA(0,2,1)(1,0,1)4  

## 최적의 파라미터 찾기 (20개 랜덤, 그리드 서치)

In [13]:
import numpy as np
import pandas as pd
from collections import defaultdict
import warnings
import ast
warnings.filterwarnings("ignore")

# 필요한 라이브러리 임포트
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesResampler
from pmdarima.arima import ARIMA

def grid_search_sarima(time_series, m=4):
    """
    주어진 시계열 데이터에 대해 후보 ARIMA 파라미터 조합에 따른 그리드 서치를 수행하고,
    각 후보 조합과 AIC 값을 출력한 후, 가장 낮은 AIC 값을 가진 파라미터 조합을 반환.
    
    후보 범위:
      - p: 0, 1, 2
      - d: 0, 1
      - q: 0, 1, 2
      - P: 0, 1
      - D: 0, 1
      - Q: 0, 1
      - m: 고정 (여기서는 4)
      
    Returns:
      best_order (tuple), best_seasonal_order (tuple), best_aic (float)
    """
    p_values = [0, 1, 2]
    d_values = [0, 1]
    q_values = [0, 1, 2]
    P_values = [0, 1]
    D_values = [0, 1]
    Q_values = [0, 1]

    best_aic = np.inf
    best_order = None
    best_seasonal_order = None

    for p in p_values:
        for d in d_values:
            for q in q_values:
                for P in P_values:
                    for D in D_values:
                        for Q in Q_values:
                            order = (p, d, q)
                            seasonal_order = (P, D, Q, m)
                            try:
                                model = ARIMA(order=order, seasonal_order=seasonal_order).fit(time_series)
                                candidate_aic = model.aic()
                                print(f"Candidate ARIMA{order}x{seasonal_order} -> AIC: {candidate_aic:.2f}")
                                if candidate_aic < best_aic:
                                    best_aic = candidate_aic
                                    best_order = order
                                    best_seasonal_order = seasonal_order
                            except Exception as e:
                                continue
    return best_order, best_seasonal_order, best_aic

# --- 최종 클러스터링 결과에 초기 파라미터를 할당하는 함수 (재사용) ---
def assign_fixed_parameters_to_clusters(cluster_ids, best_order, best_seasonal_order):
    """
    클러스터별로, 미리 선정된 초기 파라미터(best_order, best_seasonal_order)를
    각 군집에 할당하는 함수.
    
    Returns:
      dict: 각 클러스터별 초기 파라미터와 군집에 속한 시계열 ID 목록.
    """
    cluster_params = {}
    for cluster, ids in cluster_ids.items():
        cluster_params[cluster] = {
            'order': best_order,
            'seasonal_order': best_seasonal_order,
            'num_series': len(ids),
            'sample_ids': ids[:5]
        }
    return cluster_params

# =============================================================================
# 실행 코드 블럭: 도메인별 그룹화, 클러스터링, 각 군집별 그리드 서치를 통한 초기 파라미터 선정,
# 그리고 최종 결과 도출
# =============================================================================

domain_results = {}
for domain, ts_dict in log_transformed_series.items():
    print(f"\n========== 도메인: {domain} ==========")

    # 먼저 도메인 전체에 대해 DTW 기반 클러스터링을 수행하여 군집을 구분
    domain_series = []
    domain_ids = []
    for ts_id, (start_time, values) in ts_dict.items():
        if len(values) >= 8:
            domain_series.append(values)
            domain_ids.append(ts_id)
    domain_initial_length = int(np.median([len(s) for s in domain_series]))
    resampler_domain = TimeSeriesResampler(sz=domain_initial_length)
    X_domain = resampler_domain.fit_transform(domain_series)
    clustering_model = TimeSeriesKMeans(n_clusters=3, metric="dtw", max_iter=10, random_state=42, verbose=True)
    y_domain = clustering_model.fit_predict(X_domain)
    clusters = defaultdict(list)
    cluster_ids = defaultdict(list)
    for ts_id, cluster in zip(domain_ids, y_domain):
        clusters[cluster].append(ts_dict[ts_id])
        cluster_ids[cluster].append(ts_id)
    for cluster in clusters:
        print(f"클러스터 {cluster}: {len(clusters[cluster])}개 시계열")

    # 각 군집별로 랜덤 샘플 20개(20개 미만이면 전체)를 선택하여, 
    # grid search를 수행하고 그 결과 중 AIC가 가장 낮은 파라미터를 해당 군집의 초기 파라미터로 결정
    cluster_initial_params = {}  # 각 군집별 초기 파라미터 저장
    for cluster in clusters:
        cluster_data = clusters[cluster]
        id_list = cluster_ids[cluster]
        if len(cluster_data) >= 20:
            sample_indices = np.random.choice(range(len(cluster_data)), size=20, replace=False)
        else:
            sample_indices = list(range(len(cluster_data)))
        print(f"\n클러스터 {cluster}에서 {len(sample_indices)}개 샘플 선택 (전체 {len(cluster_data)}개)")

        best_cluster_aic = np.inf
        best_cluster_order = None
        best_cluster_seasonal_order = None
        # 각 샘플에 대해 grid search 수행
        for idx in sample_indices:
            ts_sample = cluster_data[idx][1]  # (start_timestamp, values)
            print(f"샘플 {id_list[idx]}에 대한 그리드 서치:")
            order_candidate, seasonal_candidate, aic_candidate = grid_search_sarima(ts_sample, m=4)
            print(f"  후보: ARIMA{order_candidate}x{seasonal_candidate} -> AIC: {aic_candidate:.2f}")
            if aic_candidate < best_cluster_aic:
                best_cluster_aic = aic_candidate
                best_cluster_order = order_candidate
                best_cluster_seasonal_order = seasonal_candidate
        print(f"클러스터 {cluster} 최종 초기 파라미터: ARIMA{best_cluster_order}x{best_cluster_seasonal_order} (AIC: {best_cluster_aic:.2f})")
        cluster_initial_params[cluster] = {
            'order': best_cluster_order,
            'seasonal_order': best_cluster_seasonal_order,
            'best_aic': best_cluster_aic
        }

    # 최종적으로, 각 클러스터에 대해 선정된 초기 파라미터를 할당
    fixed_params = assign_fixed_parameters_to_clusters(cluster_ids,
                                                       best_order=cluster_initial_params[0]['order'] if 0 in cluster_initial_params else None,
                                                       best_seasonal_order=cluster_initial_params[0]['seasonal_order'] if 0 in cluster_initial_params else None)
    # (예시로 클러스터 0의 파라미터를 할당했지만, 실제로는 각 클러스터별로 따로 저장되어야 함)
    # 여기서는 각 군집별로 최적 파라미터를 별도로 저장하는 dictionary를 생성합니다.
    fixed_params = {}
    for cluster, params in cluster_initial_params.items():
        fixed_params[cluster] = {
            'order': params['order'],
            'seasonal_order': params['seasonal_order'],
            'num_series': len(cluster_ids[cluster]),
            'sample_ids': cluster_ids[cluster][:5]
        }
    domain_results[domain] = {
        'cluster_params': fixed_params,
        'ts_cluster_map': {ts_id: cluster for cluster, ids in cluster_ids.items() for ts_id in ids}
    }

# 최종 결과 출력: 도메인별 초기 SARIMA 파라미터 요약
print("\n=== 도메인별 초기 SARIMA 파라미터 최종 결과 ===")
for domain, res in domain_results.items():
    print(f"\n도메인 {domain}:")
    for cluster, params in res['cluster_params'].items():
        print(f"  클러스터 {cluster}: {params}")

In [14]:
domain_results

In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import time
import warnings
from tqdm import tqdm
import concurrent.futures
import os
import pickle
import gc  # 가비지 컬렉션 모듈 추가
import logging

# 시계열 예측 모델들
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import SimpleExpSmoothing
from prophet import Prophet
from tbats import TBATS
from statsmodels.tsa.stattools import acf, pacf

# 평가 지표
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

# 경고 제외
warnings.filterwarnings("ignore")
# matplotlib 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')

# 결과 저장 디렉토리
RESULTS_DIR = './model_results'
os.makedirs(RESULTS_DIR, exist_ok=True)

# 도메인별 SARIMA 파라미터 설정
DOMAIN_SARIMA_PARAMS = {
    'Macro': {'order': (2, 1, 1), 'seasonal_order': (0, 1, 0, 4)},
    'Micro': {'order': (2, 1, 1), 'seasonal_order': (0, 1, 0, 4)},
    'Demographic': {'order': (0, 1, 0), 'seasonal_order': (0, 0, 0, 4)},
    'Industry': {'order': (0, 1, 0), 'seasonal_order': (0, 1, 2, 4)},
    'Finance': {'order': (2, 2, 1), 'seasonal_order': (0, 0, 0, 4)},
    'Other': {'order': (0, 2, 1), 'seasonal_order': (1, 0, 1, 4)},
    'unknown': {'order': (2, 1, 1), 'seasonal_order': (1, 1, 1, 4)}  # 기본값
}

# Theta 모델 구현 (statsmodels에서 직접 제공하지 않음)
def theta_model(y, h=48):
    """
    Theta 모델 구현
    
    Parameters:
    y (array-like): 입력 시계열 데이터
    h (int): 예측할 기간 수
    
    Returns:
    array: 예측값 배열
    """
    y = np.asarray(y)
    n = len(y)
    
    # 시계열 분해
    t = np.arange(1, n+1)
    X = np.column_stack((np.ones(n), t))
    beta = np.linalg.lstsq(X, y, rcond=None)[0]
    
    # 추세 성분 및 잔차 추출
    trend = X @ beta
    resid = y - trend
    
    # SES를 사용한 잔차 예측
    ses_model = SimpleExpSmoothing(resid).fit(optimized=True)
    resid_forecast = ses_model.forecast(h)
    
    # 추세 예측
    t_new = np.arange(n+1, n+h+1)
    X_new = np.column_stack((np.ones(h), t_new))
    trend_forecast = X_new @ beta
    
    # 최종 예측: 추세 + 잔차
    forecast = trend_forecast + resid_forecast
    
    return forecast

# 메모리 사용량 최적화를 위한 함수
def optimize_result_memory(result):
    """결과 객체에서 메모리를 많이 차지하는 부분 최적화"""
    if 'results' in result:
        for model_name, model_info in result['results'].items():
            # 모델 객체 제거 (예측에 필요 없음)
            if 'model' in model_info:
                model_info['model'] = None
            
            # forecast 데이터를 numpy에서 list로 변환 (더 효율적)
            if 'forecast' in model_info and hasattr(model_info['forecast'], 'tolist'):
                model_info['forecast'] = model_info['forecast'].tolist()
    return result

# 모델 성능 평가 함수
def evaluate_forecast(actual, predicted, naive_forecast=None):
    """
    예측 성능 평가
    
    Parameters:
    actual (array-like): 실제 값
    predicted (array-like): 예측 값
    naive_forecast (array-like, optional): 단순 예측 값 (OWA 계산용)
    
    Returns:
    dict: 평가 지표
    """
    actual = np.array(actual)
    predicted = np.array(predicted)
    
    # 결측값 처리
    mask = ~np.isnan(actual) & ~np.isnan(predicted)
    actual = actual[mask]
    predicted = predicted[mask]
    
    # 모든 값이 0인 경우 처리
    if len(actual) == 0 or len(predicted) == 0:
        return {
            'smape': np.nan,
            'mase': np.nan,
            'owa': np.nan
        }
    
    # sMAPE 계산
    smape = np.mean(200.0 * np.abs(actual - predicted) / (np.abs(actual) + np.abs(predicted))) if np.any(np.abs(actual) + np.abs(predicted) > 0) else np.nan
    
    # MASE 계산 (naive 예측: t-1 시점의 값을 사용)
    # 훈련 데이터에 대한 정보가 없으므로 단순화된 버전 사용
    if len(actual) >= 2:
        naive_errors = np.abs(np.diff(actual))
        mean_naive_error = np.mean(naive_errors) if len(naive_errors) > 0 else np.inf
        if mean_naive_error > 0:
            mase = np.mean(np.abs(actual - predicted)) / mean_naive_error
        else:
            mase = np.nan
    else:
        mase = np.nan
    
    # OWA 계산 (M4 경진대회 지표: sMAPE와 MASE의 평균)
    # OWA = 0.5 * (sMAPE / sMAPE_naive + MASE)
    # 여기서는 단순화하여 OWA = 0.5 * (smape + mase)로 계산
    if not np.isnan(smape) and not np.isnan(mase):
        owa = 0.5 * (smape + mase)
    else:
        owa = np.nan
    
    return {
        'smape': smape,
        'mase': mase,
        'owa': owa
    }

# 각 모델 구현
class ForecastingModels:
    def __init__(self, train_data, test_data, seasonal_period=4, ts_id=None, optimal_params=None, domain=None):
        """
        예측 모델 클래스
        
        Parameters:
        train_data (pd.Series): 훈련 데이터
        test_data (pd.Series): 테스트 데이터
        seasonal_period (int): 계절성 주기 (기본값: 4)
        ts_id (str): 시계열 ID (클러스터링 기반 SARIMA용)
        optimal_params (dict): 도메인별 클러스터 SARIMA 파라미터 결과
        domain (str): 시계열이 속한 도메인
        """
        self.train_data = train_data
        self.test_data = test_data
        self.h = len(test_data)
        self.seasonal_period = seasonal_period
        self.ts_id = ts_id
        self.optimal_params = optimal_params
        self.domain = domain
        self.results = {}
    
    def fit_ses(self):
        """Simple Exponential Smoothing"""
        start_time = time.time()
        try:
            model = SimpleExpSmoothing(self.train_data).fit(optimized=True)
            forecast = model.forecast(self.h)
            aic = model.aic if hasattr(model, 'aic') else np.nan
            bic = model.bic if hasattr(model, 'bic') else np.nan
            
            metrics = evaluate_forecast(self.test_data, forecast)
            metrics.update({'aic': aic, 'bic': bic})
            
            self.results['SES'] = {
                'forecast': forecast,
                'metrics': metrics,
                'model': model,
                'time': time.time() - start_time
            }
            return True
        except Exception as e:
            print(f"SES 모델 오류: {e}")
            self.results['SES'] = {
                'forecast': np.array([np.nan] * self.h),
                'metrics': {k: np.nan for k in ['smape', 'mase', 'owa', 'aic', 'bic']},
                'model': None,
                'time': time.time() - start_time,
                'error': str(e)
            }
            return False
    
    def fit_theta(self):
        """Theta 모델"""
        start_time = time.time()
        try:
            forecast = theta_model(self.train_data.values, h=self.h)
            
            metrics = evaluate_forecast(self.test_data, forecast)
            # Theta 모델은 AIC/BIC가 없음
            metrics.update({'aic': np.nan, 'bic': np.nan})
            
            self.results['Theta'] = {
                'forecast': forecast,
                'metrics': metrics,
                'model': None,  # Theta 모델은 별도의 모델 객체가 없음
                'time': time.time() - start_time
            }
            return True
        except Exception as e:
            print(f"Theta 모델 오류: {e}")
            self.results['Theta'] = {
                'forecast': np.array([np.nan] * self.h),
                'metrics': {k: np.nan for k in ['smape', 'mase', 'owa', 'aic', 'bic']},
                'model': None,
                'time': time.time() - start_time,
                'error': str(e)
            }
            return False
    
    def fit_tbats(self):
        """TBATS 모델"""
        start_time = time.time()
        try:
            # TBATS 모델은 계산 비용이 높으므로 시간 제한 설정
            model = TBATS(seasonal_periods=[self.seasonal_period], 
                         use_arma_errors=False,  # 단순화를 위해 ARMA 오차 사용 안함
                         use_box_cox=False)      # 단순화를 위해 Box-Cox 변환 사용 안함
            
            fitted_model = model.fit(self.train_data.values)
            forecast = fitted_model.forecast(steps=self.h)
            
            metrics = evaluate_forecast(self.test_data, forecast)
            # TBATS는 AIC만 제공
            metrics.update({
                'aic': fitted_model.aic if hasattr(fitted_model, 'aic') else np.nan,
                'bic': np.nan
            })
            
            self.results['TBATS'] = {
                'forecast': forecast,
                'metrics': metrics,
                'model': fitted_model,
                'time': time.time() - start_time
            }
            return True
        except Exception as e:
            print(f"TBATS 모델 오류: {e}")
            self.results['TBATS'] = {
                'forecast': np.array([np.nan] * self.h),
                'metrics': {k: np.nan for k in ['smape', 'mase', 'owa', 'aic', 'bic']},
                'model': None,
                'time': time.time() - start_time,
                'error': str(e)
            }
            return False
    
    def fit_hw_ets(self):
        """Holt-Winters Exponential Smoothing"""
        start_time = time.time()
        try:
            model = ExponentialSmoothing(
                self.train_data, 
                seasonal_periods=self.seasonal_period, 
                trend='add', 
                seasonal='add',
                use_boxcox=False  # 단순화를 위해 Box-Cox 변환 사용 안함
            ).fit(optimized=True)
            
            forecast = model.forecast(self.h)
            
            metrics = evaluate_forecast(self.test_data, forecast)
            metrics.update({
                'aic': model.aic if hasattr(model, 'aic') else np.nan,
                'bic': model.bic if hasattr(model, 'bic') else np.nan
            })
            
            self.results['HW_ETS'] = {
                'forecast': forecast,
                'metrics': metrics,
                'model': model,
                'time': time.time() - start_time
            }
            return True
        except Exception as e:
            print(f"Holt-Winters 모델 오류: {e}")
            self.results['HW_ETS'] = {
                'forecast': np.array([np.nan] * self.h),
                'metrics': {k: np.nan for k in ['smape', 'mase', 'owa', 'aic', 'bic']},
                'model': None,
                'time': time.time() - start_time,
                'error': str(e)
            }
            return False
    
    def fit_sarima(self):
        """도메인별 최적 파라미터를 사용한 SARIMA 모델"""
        start_time = time.time()
        try:
            # 도메인별 SARIMA 파라미터 가져오기
            domain_params = DOMAIN_SARIMA_PARAMS.get(self.domain, DOMAIN_SARIMA_PARAMS['unknown'])
            order = domain_params['order']
            seasonal_order = domain_params['seasonal_order']
            
            # SARIMA 모델 적합
            model = SARIMAX(
                self.train_data,
                order=order,             # 도메인별 설정된 비계절 부분: (p, d, q)
                seasonal_order=seasonal_order, # 도메인별 설정된 계절 부분: (P, D, Q, s)
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            
            fitted_model = model.fit(disp=False, maxiter=50, method='lbfgs')
            forecast = fitted_model.get_forecast(steps=self.h).predicted_mean
            
            metrics = evaluate_forecast(self.test_data, forecast)
            metrics.update({
                'aic': fitted_model.aic if hasattr(fitted_model, 'aic') else np.nan,
                'bic': fitted_model.bic if hasattr(fitted_model, 'bic') else np.nan,
                'order': str(order),
                'seasonal_order': str(seasonal_order)
            })
            
            self.results['SARIMA'] = {
                'forecast': forecast,
                'metrics': metrics,
                'model': fitted_model,
                'time': time.time() - start_time,
                'sarima_info': {
                    'domain': self.domain,
                    'order': order,
                    'seasonal_order': seasonal_order
                }
            }
            return True
        except Exception as e:
            print(f"SARIMA 모델 오류: {e}")
            self.results['SARIMA'] = {
                'forecast': np.array([np.nan] * self.h),
                'metrics': {k: np.nan for k in ['smape', 'mase', 'owa', 'aic', 'bic']},
                'model': None,
                'time': time.time() - start_time,
                'error': str(e)
            }
            return False
    
    def fit_cluster_sarima(self):
        """클러스터링 기반 SARIMA 모델"""
        if self.optimal_params is None or self.ts_id is None or self.domain is None:
            return False
        
        start_time = time.time()
        try:
            # 도메인이 optimal_params에 존재하는지 확인
            if self.domain in self.optimal_params:
                ts_cluster_map = self.optimal_params[self.domain]['ts_cluster_map']
                cluster_params = self.optimal_params[self.domain]['cluster_params']
                
                if self.ts_id in ts_cluster_map:
                    cluster = ts_cluster_map[self.ts_id]
                    if cluster in cluster_params:
                        # 클러스터별 최적 파라미터 가져오기
                        order = cluster_params[cluster]['order']
                        seasonal_order = cluster_params[cluster]['seasonal_order']
                        
                        # 모델 학습 및 예측
                        model = SARIMAX(
                            self.train_data, 
                            order=order, 
                            seasonal_order=seasonal_order,
                            enforce_stationarity=False,
                            enforce_invertibility=False
                        )
                        
                        fitted_model = model.fit(disp=False, maxiter=50, method='lbfgs')
                        forecast = fitted_model.get_forecast(steps=self.h).predicted_mean
                        
                        metrics = evaluate_forecast(self.test_data, forecast)
                        metrics.update({
                            'aic': fitted_model.aic if hasattr(fitted_model, 'aic') else np.nan,
                            'bic': fitted_model.bic if hasattr(fitted_model, 'bic') else np.nan,
                            'cluster': cluster,
                            'domain': self.domain
                        })
                        
                        self.results['ClusterSARIMA'] = {
                            'forecast': forecast,
                            'metrics': metrics,
                            'model': fitted_model,
                            'time': time.time() - start_time,
                            'cluster_info': {
                                'domain': self.domain,
                                'cluster': cluster,
                                'order': order,
                                'seasonal_order': seasonal_order
                            }
                        }
                        return True
            
            # 도메인이나 클러스터를 찾을 수 없는 경우 도메인별 기본 SARIMA 사용
            domain_params = DOMAIN_SARIMA_PARAMS.get(self.domain, DOMAIN_SARIMA_PARAMS['unknown'])
            order = domain_params['order']
            seasonal_order = domain_params['seasonal_order']
            
            model = SARIMAX(
                self.train_data,
                order=order,
                seasonal_order=seasonal_order,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            
            fitted_model = model.fit(disp=False, maxiter=50, method='lbfgs')
            forecast = fitted_model.get_forecast(steps=self.h).predicted_mean
            
            metrics = evaluate_forecast(self.test_data, forecast)
            metrics.update({
                'aic': fitted_model.aic if hasattr(fitted_model, 'aic') else np.nan,
                'bic': fitted_model.bic if hasattr(fitted_model, 'bic') else np.nan,
                'cluster': 'unknown',
                'domain': self.domain
            })
            
            self.results['ClusterSARIMA'] = {
                'forecast': forecast,
                'metrics': metrics,
                'model': fitted_model,
                'time': time.time() - start_time,
                'cluster_info': {
                    'domain': self.domain,
                    'cluster': 'unknown',
                    'order': order,
                    'seasonal_order': seasonal_order
                }
            }
            return True
            
        except Exception as e:
            print(f"클러스터 SARIMA 모델 오류: {e}")
            self.results['ClusterSARIMA'] = {
                'forecast': np.array([np.nan] * self.h),
                'metrics': {k: np.nan for k in ['smape', 'mase', 'owa', 'aic', 'bic']},
                'model': None,
                'time': time.time() - start_time,
                'error': str(e),
                'cluster_info': {
                    'domain': self.domain,
                    'cluster': 'error'
                }
            }
            return False
    
    def fit_prophet(self):
        """Prophet 모델"""
        start_time = time.time()
        try:
            # Prophet용 데이터프레임 준비
            df = pd.DataFrame({
                'ds': self.train_data.index,
                'y': self.train_data.values
            })
            
            model = Prophet(
                daily_seasonality=True,
                yearly_seasonality=False,
                weekly_seasonality=True,
                seasonality_mode='additive'
            )
            
            model.fit(df)
            
            # 예측 기간 준비
            future = model.make_future_dataframe(periods=self.h, freq='h')
            forecast_df = model.predict(future)
            
            # 테스트 기간에 해당하는 예측값 추출
            forecast = forecast_df.iloc[-self.h:]['yhat'].values
            
            metrics = evaluate_forecast(self.test_data, forecast)
            # Prophet은 AIC/BIC가 없음
            metrics.update({'aic': np.nan, 'bic': np.nan})
            
            self.results['Prophet'] = {
                'forecast': forecast,
                'metrics': metrics,
                'model': model,
                'time': time.time() - start_time
            }
            return True
        except Exception as e:
            print(f"Prophet 모델 오류: {e}")
            self.results['Prophet'] = {
                'forecast': np.array([np.nan] * self.h),
                'metrics': {k: np.nan for k in ['smape', 'mase', 'owa', 'aic', 'bic']},
                'model': None,
                'time': time.time() - start_time,
                'error': str(e)
            }
            return False
    
    def fit_all_models(self, optimize_memory=True, exclude_heavy_models=False, include_cluster_sarima=False):
        """
        모든 모델 학습 및 평가
        
        Parameters:
        optimize_memory (bool): 메모리 사용량 최적화 여부
        exclude_heavy_models (bool): 무거운 모델(Prophet, TBATS)을 제외할지 여부
        include_cluster_sarima (bool): 클러스터링 기반 SARIMA 모델 포함 여부
        
        Returns:
        dict: 모델 결과 딕셔너리
        """
        self.fit_ses()
        self.fit_theta()
        
        if not exclude_heavy_models:
            self.fit_tbats()
        
        self.fit_hw_ets()
        self.fit_sarima()
        
        if include_cluster_sarima:
            self.fit_cluster_sarima()
        
        if not exclude_heavy_models:
            self.fit_prophet()
        
        # 메모리 최적화 - 큰 모델 객체 제거
        if optimize_memory:
            for model_name, model_info in self.results.items():
                if 'model' in model_info:
                    # 모델 객체는 결과 분석에 필요 없으므로 제거
                    model_info['model'] = None
        
        return self.results

def evaluate_time_series(ts_id, domain, log_transformed_series, test_size=8, seasonal_period=4, 
                        optimize_memory=True, exclude_heavy_models=False, 
                        optimal_params=None, include_cluster_sarima=False):
    """
    단일 시계열에 대해 모든 모델 평가 (로그 변환된 데이터 사용)
    
    Parameters:
    ts_id (str): 시계열 ID
    domain (str): 시계열이 속한 도메인
    log_transformed_series (dict): 로그 변환된 시계열 데이터 딕셔너리
    test_size (int): 테스트 세트 크기
    seasonal_period (int): 계절성 주기 (기본값: 4)
    optimize_memory (bool): 메모리 사용량 최적화 여부
    exclude_heavy_models (bool): 무거운 모델 제외 여부
    optimal_params (dict): 도메인별 클러스터 SARIMA 파라미터
    include_cluster_sarima (bool): 클러스터링 기반 SARIMA 모델 포함 여부
    
    Returns:
    dict: 모델 평가 결과
    """
    try:
        # 로그 변환된 시계열 데이터 가져오기
        if domain not in log_transformed_series or ts_id not in log_transformed_series[domain]:
            return {
                'ts_id': ts_id,
                'domain': domain,
                'error': f"시계열 {domain}/{ts_id}가 로그 변환 데이터에 없습니다."
            }
        
        start_time, values = log_transformed_series[domain][ts_id]
        
        # 결측치 제거된 값 배열 생성
        values_clean = np.array([v for v in values if not np.isnan(v)])
        
        # 시계열 생성
        if isinstance(start_time, str):
            start_time = pd.to_datetime(start_time)
        
        date_range = pd.date_range(start=start_time, periods=len(values_clean), freq='h')
        series = pd.Series(values_clean, index=date_range)
        
        # 훈련/테스트 세트 분할
        if len(series) <= test_size:
            return {
                'ts_id': ts_id,
                'domain': domain,
                'error': f"시계열 {ts_id}의 길이({len(series)})가 테스트 크기({test_size})보다 작습니다."
            }
            
        train_size = len(series) - test_size
        train_data = series[:train_size]
        test_data = series[train_size:]
        
        # 모델 학습 및 평가
        models = ForecastingModels(
            train_data, test_data, 
            seasonal_period=seasonal_period,
            ts_id=ts_id,
            optimal_params=optimal_params,
            domain=domain
        )
        
        results = models.fit_all_models(
            optimize_memory=optimize_memory, 
            exclude_heavy_models=exclude_heavy_models,
            include_cluster_sarima=include_cluster_sarima
        )
        
        result_dict = {
            'ts_id': ts_id,
            'domain': domain,
            'results': results,
            'train_data': train_data,
            'test_data': test_data
        }
        
        # 메모리 최적화
        if optimize_memory:
            # 훈련/테스트 데이터를 numpy 배열로 변환하여 메모리 사용량 감소
            result_dict['train_data'] = train_data.values
            result_dict['test_data'] = test_data.values
        
        return result_dict
    except Exception as e:
        print(f"시계열 {domain}/{ts_id} 평가 중 오류 발생: {e}")
        return {
            'ts_id': ts_id,
            'domain': domain,
            'error': str(e)
        }

# 모델 결과 요약 함수
def summarize_model_results(all_results):
    """
    모든 시계열에 대한 모델 결과 요약
    
    Parameters:
    all_results (list): 각 시계열의 모델 결과 리스트
    
    Returns:
    pd.DataFrame: 요약 데이터프레임
    """
    summary_data = []
    
    for ts_result in all_results:
        if 'error' in ts_result:
            continue
        
        ts_id = ts_result['ts_id']
        domain = ts_result.get('domain', 'unknown')  # 도메인 정보 추가
        results = ts_result['results']
        
        for model_name, model_result in results.items():
            metrics = model_result['metrics']
            execution_time = model_result['time']
            
            row = {
                'ts_id': ts_id,
                'domain': domain,  # 도메인 정보 추가
                'model': model_name,
                'smape': metrics['smape'],
                'mase': metrics['mase'],
                'owa': metrics['owa'],
                'aic': metrics['aic'],
                'bic': metrics['bic'],
                'execution_time': execution_time
            }
            
            # SARIMA 모델인 경우 파라미터 정보 추가
            if model_name == 'SARIMA' and 'sarima_info' in model_result:
                row['order'] = str(model_result['sarima_info'].get('order', ''))
                row['seasonal_order'] = str(model_result['sarima_info'].get('seasonal_order', ''))
                
            # 클러스터 SARIMA 모델인 경우 클러스터 정보 추가
            # 클러스터 SARIMA 모델인 경우 클러스터 정보 추가
            if model_name == 'ClusterSARIMA' and 'cluster_info' in model_result:
                row['cluster'] = model_result['cluster_info'].get('cluster', 'unknown')
                row['order'] = str(model_result['cluster_info'].get('order', ''))
                row['seasonal_order'] = str(model_result['cluster_info'].get('seasonal_order', ''))
                
            summary_data.append(row)
    
    summary_df = pd.DataFrame(summary_data)
    return summary_df

# 개별 시계열에 대한 모델 예측 시각화 함수 개선
def visualize_forecast_with_intervals(ts_id, ts_result, log_transformed_series=None):
   """
   개별 시계열에 대한 모델 예측을 시각화하고 가능한 경우 신뢰구간 추가
   
   Parameters:
   ts_id (str): 시계열 ID
   ts_result (dict): 시계열 평가 결과
   log_transformed_series (dict, optional): 로그 변환된 시계열 데이터 딕셔너리
   """
   # 한글 폰트 설정 - 이 함수 내에서만 적용
   import matplotlib as mpl
   import matplotlib.font_manager as fm
   import platform
   
   # 스타일 설정
   plt.style.use('seaborn-v0_8-whitegrid')
   
   # 시스템별 폰트 설정
   system_name = platform.system()
   if system_name == "Windows":
       plt.rcParams['font.family'] = 'Malgun Gothic'
   elif system_name == "Darwin":  # macOS
       plt.rcParams['font.family'] = 'AppleGothic'
   else:  # Linux 등
       # 범용 폰트 사용
       plt.rcParams['font.family'] = 'NanumGothic, DejaVu Sans'
   
   # 마이너스 기호 깨짐 방지
   mpl.rcParams['axes.unicode_minus'] = False
   
   if not ts_result or 'error' in ts_result:
       print(f"시계열 {ts_id}에 대한 결과가 없거나 오류가 있습니다.")
       return
       
   train_data = ts_result['train_data']
   test_data = ts_result['test_data']
   results = ts_result['results']
   domain = ts_result.get('domain', 'unknown')  # 도메인 정보 추가
   
   # 데이터가 numpy 배열인 경우 Series로 변환
   if isinstance(train_data, np.ndarray):
       # 원본 데이터의 시작 시간 가져오기 (가능한 경우)
       start_time = datetime(2015, 1, 1)  # 기본값
       if log_transformed_series and domain in log_transformed_series and ts_id in log_transformed_series[domain]:
           start_time = log_transformed_series[domain][ts_id][0]
           if isinstance(start_time, str):
               start_time = pd.to_datetime(start_time)
           
       date_range = pd.date_range(start=start_time, periods=len(train_data), freq='h')
       train_data = pd.Series(train_data, index=date_range)
       test_data = pd.Series(test_data, index=pd.date_range(start=date_range[-1] + timedelta(hours=1), periods=len(test_data), freq='h'))
   
   # 그래프 설정
   plt.figure(figsize=(16, 10))
   
   # 서브플롯 - 전체 데이터와 테스트 기간 확대뷰
   fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12), gridspec_kw={'height_ratios': [2, 3]})
   
   # 1. 전체 데이터 뷰 (ax1)
   # 원본 데이터
   ax1.plot(train_data.index, train_data, label='훈련 데이터', color='black', linewidth=1.5, alpha=0.7)
   ax1.plot(test_data.index, test_data, label='테스트 데이터', color='blue', linewidth=2)
   
   # 테스트 영역 표시
   ax1.axvspan(test_data.index[0], test_data.index[-1], alpha=0.1, color='blue', label='테스트 구간')
   
   # 각 모델의 예측
   # 시각적으로 더 구분하기 쉬운 색상 팔레트 사용
   colors = plt.cm.Set2(np.linspace(0, 1, 8))  # 뚜렷하게 구분되는 색상 팔레트
   
   # 신뢰구간을 표시할 모델 (일반적으로 SARIMA와 Prophet은 신뢰구간 제공)
   models_with_intervals = ['SARIMA', 'ClusterSARIMA', 'Prophet']
   
   for i, (model_name, model_result) in enumerate(results.items()):
       forecast = model_result['forecast']
       metrics = model_result['metrics']
       
       # forecast가 리스트인 경우 numpy 배열로 변환
       if isinstance(forecast, list):
           forecast = np.array(forecast)
       
       color = colors[i % len(colors)]
       linestyle = ['-', '--', '-.', ':'][i % 4]  # 다양한 선 스타일 사용
       
       # 모델 이름과 성능 지표를 포함한 범례
       label = f'{model_name} (OWA: {metrics["owa"]:.2f})'
       
       ax1.plot(test_data.index, forecast, 
                label=label, 
                color=color, linewidth=2, linestyle=linestyle)
   
   ax1.set_title(f'시계열 {ts_id} - {domain} 도메인 - 전체 데이터 및 예측', fontsize=16)  # 도메인 정보 추가
   ax1.set_xlabel('시간', fontsize=12)
   ax1.set_ylabel('로그 값', fontsize=12)  # 로그 변환 데이터임을 표시
   ax1.legend(loc='best', fontsize=10)
   ax1.grid(True, alpha=0.3)
   
   # 2. 테스트 기간 확대뷰 (ax2)
   # 테스트 기간 + 약간의 여백
   padding = len(test_data) // 4  # 테스트 기간의 1/4만큼 여백
   if padding > 0 and len(train_data) > padding:
       start_idx = -padding - len(test_data)
       ax2.plot(train_data.index[start_idx:], train_data.iloc[start_idx:], 
               label='훈련 데이터', color='black', linewidth=1.5, alpha=0.7)
   else:
       ax2.plot(train_data.index, train_data, 
               label='훈련 데이터', color='black', linewidth=1.5, alpha=0.7)
   
   ax2.plot(test_data.index, test_data, label='테스트 데이터', color='blue', linewidth=2.5)
   
   # 테스트 구간 경계선 표시
   ax2.axvline(test_data.index[0], linestyle='--', color='blue', alpha=0.7)
   
   # 각 모델의 예측 및 신뢰구간 (가능한 경우)
   for i, (model_name, model_result) in enumerate(results.items()):
       forecast = model_result['forecast']
       
       # forecast가 리스트인 경우 numpy 배열로 변환
       if isinstance(forecast, list):
           forecast = np.array(forecast)
       
       color = colors[i % len(colors)]
       linestyle = ['-', '--', '-.', ':'][i % 4]
       
       # 더 두꺼운 선으로 표시하여 구분 용이하게
       ax2.plot(test_data.index, forecast, 
               color=color, linewidth=2.5, linestyle=linestyle,
               label=f'{model_name} (OWA: {model_result["metrics"]["owa"]:.2f})')
       
       # 신뢰구간 표시 (특정 모델에 대해)
       if model_name in models_with_intervals:
           try:
               # SARIMA 모델 신뢰구간
               if model_name in ['SARIMA', 'ClusterSARIMA']:
                   # 모델 정보 가져오기
                   if model_name == 'SARIMA' and 'sarima_info' in model_result:
                       order = model_result['sarima_info'].get('order', (2, 1, 1))
                       seasonal_order = model_result['sarima_info'].get('seasonal_order', (1, 1, 1, 4))
                   elif model_name == 'ClusterSARIMA' and 'cluster_info' in model_result:
                       order = model_result['cluster_info'].get('order', (2, 1, 1))
                       seasonal_order = model_result['cluster_info'].get('seasonal_order', (1, 1, 1, 4))
                   else:
                       # 도메인별 기본값 사용
                       domain_params = DOMAIN_SARIMA_PARAMS.get(domain, DOMAIN_SARIMA_PARAMS['unknown'])
                       order = domain_params['order']
                       seasonal_order = domain_params['seasonal_order']
                   
                   # 모델 재구성 및 신뢰구간 계산
                   temp_model = SARIMAX(
                       train_data,
                       order=order,
                       seasonal_order=seasonal_order,
                       enforce_stationarity=False,
                       enforce_invertibility=False
                   ).fit(disp=False)
                   
                   forecast_obj = temp_model.get_forecast(steps=len(test_data))
                   conf_int = forecast_obj.conf_int(alpha=0.05)  # 95% 신뢰구간
                   
                   # 신뢰구간 표시
                   ax2.fill_between(test_data.index, 
                                   conf_int.iloc[:, 0], 
                                   conf_int.iloc[:, 1], 
                                   color=color, alpha=0.2)
               
               # Prophet 모델 신뢰구간 (별도로 계산)
               elif model_name == 'Prophet':
                   if 'forecast_df' in model_result and model_result['forecast_df'] is not None:
                       # 저장된 forecast_df가 있는 경우
                       prophet_forecast = model_result['forecast_df']
                       ax2.fill_between(test_data.index,
                                       prophet_forecast['yhat_lower'].values[-len(test_data):],
                                       prophet_forecast['yhat_upper'].values[-len(test_data):],
                                       color=color, alpha=0.2)
                   else:
                       # 없는 경우 근사적 계산
                       residuals = test_data.values - forecast
                       std_resid = np.std(residuals)
                       z_value = 1.96  # 95% 신뢰수준의 z값
                       
                       lower_ci = forecast - z_value * std_resid
                       upper_ci = forecast + z_value * std_resid
                       
                       ax2.fill_between(test_data.index,
                                      lower_ci,
                                      upper_ci,
                                      color=color, alpha=0.2)
           except Exception as e:
               print(f"{model_name} 모델 신뢰구간 계산 오류: {e}")
       
       # 모든 모델에 대해 신뢰구간 계산 (신뢰구간이 없는 모델도 포함)
       else:
           try:
               # 잔차 기반 근사적 신뢰구간 계산
               residuals = test_data.values - forecast
               std_resid = np.std(residuals)
               z_value = 1.96  # 95% 신뢰수준의 z값
               
               lower_ci = forecast - z_value * std_resid
               upper_ci = forecast + z_value * std_resid
               
               ax2.fill_between(test_data.index,
                              lower_ci,
                              upper_ci,
                              color=color, alpha=0.2)
           except Exception as e:
               print(f"{model_name} 근사 신뢰구간 계산 오류: {e}")
   
   ax2.set_title(f'시계열 {ts_id} - {domain} 도메인 - 테스트 기간 확대', fontsize=16)  # 도메인 정보 추가
   ax2.set_xlabel('시간', fontsize=12)
   ax2.set_ylabel('로그 값', fontsize=12)  # 로그 변환 데이터임을 표시
   
   # 레이아웃 개선 - 더 명확한 범례
   handles, labels = ax2.get_legend_handles_labels()
   ax2.legend(handles, labels, loc='best', fontsize=10, framealpha=0.8, 
             bbox_to_anchor=(1, 1), title='모델 및 성능')
   
   ax2.grid(True, alpha=0.3)
   
   # 모델 간 더 명확한 비교를 위한 오차 표시
   model_errors = {}
   for model_name, model_result in results.items():
       forecast = model_result['forecast']
       
       # forecast가 리스트인 경우 numpy 배열로 변환
       if isinstance(forecast, list):
           forecast = np.array(forecast)
       
       # 모델별 RMSE, MAE 계산
       rmse = np.sqrt(np.mean((test_data.values - forecast) ** 2))
       mae = np.mean(np.abs(test_data.values - forecast))
       model_errors[model_name] = {'RMSE': rmse, 'MAE': mae}
   
   # 오차 정보를 텍스트로 표시
   error_text = "모델별 오차:\n"
   for model, errors in sorted(model_errors.items(), key=lambda x: x[1]['RMSE']):
       error_text += f"{model}: RMSE={errors['RMSE']:.2f}, MAE={errors['MAE']:.2f}\n"
   
   plt.figtext(0.02, 0.02, error_text, fontsize=10, 
              bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'))
   
   plt.tight_layout()
   plt.savefig(os.path.join(RESULTS_DIR, f'forecast_detailed_{domain}_{ts_id}.png'), dpi=150)  # 도메인 정보 추가
   plt.show()

   # 모델별 잔차 분석 그래프 - 추가적인 정보 제공
   plt.figure(figsize=(16, 10))
   fig, axes = plt.subplots(len(results), 1, figsize=(16, 4*len(results)))
   
   if len(results) == 1:
       axes = [axes]  # 단일 모델 경우 리스트로 변환
   
   for i, (model_name, model_result) in enumerate(results.items()):
       forecast = model_result['forecast']
       
       # forecast가 리스트인 경우 numpy 배열로 변환
       if isinstance(forecast, list):
           forecast = np.array(forecast)
       
       # 잔차 계산
       residuals = test_data.values - forecast
       
       # 잔차 그래프
       ax = axes[i]
       ax.plot(test_data.index, residuals, marker='o', linestyle='None', 
              color=colors[i % len(colors)], alpha=0.7, markersize=4)
       ax.axhline(y=0, color='r', linestyle='-', alpha=0.3)
       
       # 이동 평균 추가
       window = min(5, len(residuals) // 3) if len(residuals) > 5 else 1
       if window > 1:
           rolling_mean = pd.Series(residuals).rolling(window=window).mean()
           ax.plot(test_data.index, rolling_mean, color='black', linewidth=2, 
                 label=f'{window}-포인트 이동 평균')
       
       ax.set_title(f'{model_name} 잔차', fontsize=14)
       ax.set_ylabel('잔차 (실제 - 예측)', fontsize=10)
       
       # 잔차 통계 추가
       mean_resid = np.mean(residuals)
       std_resid = np.std(residuals)
       
       stats_text = f"평균: {mean_resid:.2f}\n표준편차: {std_resid:.2f}"
       ax.text(0.02, 0.90, stats_text, transform=ax.transAxes,
              bbox=dict(facecolor='white', alpha=0.8))
       
       if window > 1:
           ax.legend(loc='best')
   
   plt.tight_layout()
   plt.savefig(os.path.join(RESULTS_DIR, f'residuals_{domain}_{ts_id}.png'), dpi=150)  # 도메인 정보 추가
   plt.show()
   
   return fig

# 결과 시각화 함수 업데이트
def visualize_results(all_results, log_transformed_series=None, top_n=5):
   """
   결과 시각화 - 개선된 시각화 적용
   
   Parameters:
   all_results (list): 각 시계열의 모델 결과 리스트
   log_transformed_series (dict): 로그 변환된 시계열 데이터 딕셔너리
   top_n (int): 시각화할 상위 시계열 수
   """
   summary_df = summarize_model_results(all_results)
   
   # 1. 모델별 평균 성능 비교 (전체 및 도메인별)
   metrics = ['smape', 'mase', 'owa']
   
   # 전체 성능
   plt.figure(figsize=(15, 15))
   
   for i, metric in enumerate(metrics):
       plt.subplot(len(metrics), 1, i+1)
       
       model_avg = summary_df.groupby('model')[metric].mean().sort_values()
       ax = sns.barplot(x=model_avg.index, y=model_avg.values, palette='viridis')
       
       # 값 표시 추가
       for j, v in enumerate(model_avg.values):
           ax.text(j, v + 0.01, f"{v:.3f}", ha='center', fontsize=10)
       
       plt.title(f'모델별 평균 {metric.upper()} (낮을수록 좋음)', fontsize=14)
       plt.xlabel('모델', fontsize=12)
       plt.ylabel(metric.upper(), fontsize=12)
       plt.xticks(rotation=45)
       plt.grid(True, alpha=0.3)
   
   plt.tight_layout()
   plt.savefig(os.path.join(RESULTS_DIR, 'model_performance_comparison.png'), dpi=150)
   plt.show()
   
   # 도메인별 성능 비교
   for metric in metrics:
       plt.figure(figsize=(15, 10))
       
       for i, domain in enumerate(summary_df['domain'].unique()):
           domain_df = summary_df[summary_df['domain'] == domain]
           model_avg = domain_df.groupby('model')[metric].mean().sort_values()
           
           plt.subplot(len(summary_df['domain'].unique()), 1, i+1)
           ax = sns.barplot(x=model_avg.index, y=model_avg.values, palette='viridis')
           
           # 값 표시 추가
           for j, v in enumerate(model_avg.values):
               ax.text(j, v + 0.01, f"{v:.3f}", ha='center', fontsize=10)
           
           plt.title(f'{domain} 도메인 - 모델별 평균 {metric.upper()}', fontsize=14)
           plt.xlabel('모델', fontsize=12)
           plt.ylabel(metric.upper(), fontsize=12)
           plt.xticks(rotation=45)
           plt.grid(True, alpha=0.3)
       
       plt.tight_layout()
       plt.savefig(os.path.join(RESULTS_DIR, f'domain_model_performance_{metric}.png'), dpi=150)
       plt.show()
   
   # 2. 모델별 실행 시간 비교
   plt.figure(figsize=(14, 10))
   
   # 2개의 서브플롯 생성 - 일반 스케일과 로그 스케일
   fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
   
   model_times = summary_df.groupby('model')['execution_time'].mean().sort_values()
   
   # 일반 스케일
   sns.barplot(x=model_times.index, y=model_times.values, palette='magma', ax=ax1)
   
   # 값 표시 추가
   for i, v in enumerate(model_times.values):
       ax1.text(i, v + 0.1, f"{v:.3f}초", ha='center', fontsize=9)
   
   ax1.set_title('모델별 평균 실행 시간 (초)', fontsize=14)
   ax1.set_xlabel('모델', fontsize=12)
   ax1.set_ylabel('실행 시간 (초)', fontsize=12)
   ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)
   ax1.grid(True, alpha=0.3)
   
   plt.tight_layout()
   plt.savefig(os.path.join(RESULTS_DIR, 'model_execution_times.png'), dpi=150)
   plt.show()
   
   # 3. 개별 시계열에 대한 모델 예측 시각화 (도메인별 상위 N개 시계열)
   # 도메인별로 상위 시계열 선택
   for domain in summary_df['domain'].unique():
       domain_df = summary_df[summary_df['domain'] == domain]
       best_ts_ids = domain_df.groupby('ts_id')['owa'].min().sort_values()[:max(1, int(top_n/len(summary_df['domain'].unique())))].index
       
       for ts_id in best_ts_ids:
           ts_result = next((r for r in all_results if r['ts_id'] == ts_id), None)
           if not ts_result or 'error' in ts_result:
               continue
               
           # 개선된 시각화 함수 호출
           visualize_forecast_with_intervals(ts_id, ts_result, log_transformed_series)
   
   # 4. 박스플롯: 모델별 성능 분포 (도메인별)
   for domain in summary_df['domain'].unique():
       domain_df = summary_df[summary_df['domain'] == domain]
       
       plt.figure(figsize=(15, 15))
       
       for i, metric in enumerate(metrics):
           plt.subplot(len(metrics), 1, i+1)
           
           sns.boxplot(x='model', y=metric, data=domain_df, palette='viridis')
           
           # 평균값 표시 추가
           means = domain_df.groupby('model')[metric].mean()
           pos = range(len(means))
           for j, m in zip(pos, means):
               plt.text(j, m, f"{m:.3f}", ha='center', va='bottom', fontsize=9, color='black')
           
           plt.title(f'{domain} 도메인 - 모델별 {metric.upper()} 분포', fontsize=14)
           plt.xlabel('모델', fontsize=12)
           plt.ylabel(metric.upper(), fontsize=12)
           plt.xticks(rotation=45)
           plt.grid(True, alpha=0.3)
       
       plt.tight_layout()
       plt.savefig(os.path.join(RESULTS_DIR, f'model_performance_distributions_{domain}.png'), dpi=150)
       plt.show()
   
   # 5. 클러스터 SARIMA와 일반 SARIMA 비교 (도메인별)
   if 'ClusterSARIMA' in summary_df['model'].values and 'SARIMA' in summary_df['model'].values:
       for domain in summary_df['domain'].unique():
           domain_df = summary_df[summary_df['domain'] == domain]
           sarima_models = domain_df[domain_df['model'].isin(['SARIMA', 'ClusterSARIMA'])]
           
           if len(sarima_models) > 0:
               plt.figure(figsize=(12, 10))
               
               # 세 가지 성능 지표 비교
               for i, metric in enumerate(['smape', 'mase', 'owa']):
                   plt.subplot(3, 1, i+1)
                   
                   # 성능 분포 시각화
                   sns.boxplot(x='model', y=metric, data=sarima_models, palette=['skyblue', 'lightgreen'])
                   
                   # 개별 데이터 포인트 추가 - 투명도 적용
                   sns.stripplot(x='model', y=metric, data=sarima_models, color='black', alpha=0.3, size=3)
                   
                   # 평균값 표시 라인
                   for j, model in enumerate(['SARIMA', 'ClusterSARIMA']):
                       if model in sarima_models['model'].values:
                           mean_val = sarima_models[sarima_models['model'] == model][metric].mean()
                           plt.hlines(y=mean_val, xmin=j-0.3, xmax=j+0.3, colors='red', linestyles='dashed', linewidth=2)
                           plt.text(j, mean_val, f" 평균: {mean_val:.3f}", ha='left', va='center', fontsize=9)
                   
                   plt.title(f'{domain} 도메인 - 일반 SARIMA vs 클러스터 SARIMA {metric.upper()} 비교', fontsize=14)
                   plt.ylabel(metric.upper(), fontsize=12)
                   plt.grid(True, alpha=0.3)
               
               plt.tight_layout()
               plt.savefig(os.path.join(RESULTS_DIR, f'sarima_vs_cluster_sarima_{domain}.png'), dpi=150)
               plt.show()
               
               # 클러스터별 ClusterSARIMA 성능 비교 (클러스터 정보가 있는 경우)
               cluster_sarima = sarima_models[sarima_models['model'] == 'ClusterSARIMA']
               
               if len(cluster_sarima) > 0 and 'cluster' in cluster_sarima.columns and not cluster_sarima['cluster'].isna().all():
                   plt.figure(figsize=(14, 10))
                   
                   # 각 클러스터별 시계열 개수 계산
                   cluster_counts = cluster_sarima['cluster'].value_counts().sort_index()
                   
                   for i, metric in enumerate(['smape', 'mase', 'owa']):
                       plt.subplot(3, 1, i+1)
                       
                       # 클러스터별 성능 분포
                       ax = sns.boxplot(x='cluster', y=metric, data=cluster_sarima, palette='viridis')
                       
                       # 개별 데이터 포인트 추가
                       sns.stripplot(x='cluster', y=metric, data=cluster_sarima, color='black', alpha=0.3, size=3)
                       
                       # 클러스터별 시계열 개수 표시
                       for j, (cluster, count) in enumerate(cluster_counts.items()):
                           if str(cluster) in [label.get_text() for label in ax.get_xticklabels()]:
                               plt.text(j, ax.get_ylim()[1] * 0.95, f"n={count}", ha='center', va='top', fontsize=9)
                       
                       plt.title(f'{domain} 도메인 - 클러스터별 {metric.upper()} 성능 분포', fontsize=14)
                       plt.xlabel('클러스터', fontsize=12)
                       plt.ylabel(metric.upper(), fontsize=12)
                       plt.grid(True, alpha=0.3)
                   
                   plt.tight_layout()
                   plt.savefig(os.path.join(RESULTS_DIR, f'cluster_sarima_performance_{domain}.png'), dpi=150)
                   plt.show()

           # SARIMA와 ClusterSARIMA 비교 (둘 다 있는 경우)
           sarima_df = domain_df[domain_df['model'] == 'SARIMA']
           cluster_sarima_df = domain_df[domain_df['model'] == 'ClusterSARIMA']
           
           # 각 시계열별로 SARIMA와 ClusterSARIMA의 성능 차이 계산
           ts_comparison = []
           for ts_id in domain_df['ts_id'].unique():
               ts_sarima = sarima_df[sarima_df['ts_id'] == ts_id]
               ts_cluster = cluster_sarima_df[cluster_sarima_df['ts_id'] == ts_id]
               
               if len(ts_sarima) == 1 and len(ts_cluster) == 1:  # 두 모델 모두 있는 경우만 비교
                   sarima_owa = ts_sarima['owa'].values[0]
                   cluster_owa = ts_cluster['owa'].values[0]
                   
                   comparison = {
                       'ts_id': ts_id,
                       'domain': domain,
                       'sarima_owa': sarima_owa,
                       'cluster_owa': cluster_owa,
                       'improvement': (sarima_owa - cluster_owa) / sarima_owa * 100,  # 개선율(%)
                       'is_better': sarima_owa > cluster_owa  # ClusterSARIMA가 더 나은지 여부
                   }
                   
                   # 클러스터 정보 추가
                   if 'cluster' in ts_cluster.columns:
                       comparison['cluster'] = ts_cluster['cluster'].values[0]
                   
                   ts_comparison.append(comparison)
           
           comparison_df = pd.DataFrame(ts_comparison)
           
           if not comparison_df.empty:
               # 개선율 시각화
               plt.figure(figsize=(14, 12))
               
               # 1. 개선율 분포 (히스토그램 + 커널 밀도)
               plt.subplot(2, 1, 1)
               
               # 개선율 히스토그램에 컬러 구분 (양수: 개선, 음수: 악화)
               improvement_data = comparison_df['improvement']
               
               # 양수 및 음수 값 분리
               positive_mask = improvement_data >= 0
               negative_mask = ~positive_mask
               
               # 히스토그램 + 커널 밀도 그래프
               sns.histplot(improvement_data[positive_mask], kde=True, color='green', alpha=0.5, 
                          label='개선 (ClusterSARIMA가 우수)', bins=20)
               sns.histplot(improvement_data[negative_mask], kde=True, color='red', alpha=0.5, 
                          label='악화 (SARIMA가 우수)', bins=20)
               
               # 0 지점에 수직선 추가
               plt.axvline(x=0, color='black', linestyle='--', alpha=0.7)
               
               # 통계 정보 추가
               better_pct = (comparison_df['is_better'].sum() / len(comparison_df)) * 100
               mean_improvement = comparison_df['improvement'].mean()
               median_improvement = comparison_df['improvement'].median()
               
               stats_text = (
                   f"ClusterSARIMA 우수 비율: {better_pct:.1f}%\n"
                   f"평균 개선율: {mean_improvement:.2f}%\n"
                   f"중앙값 개선율: {median_improvement:.2f}%"
               )
               
               plt.text(0.02, 0.85, stats_text, transform=plt.gca().transAxes,
                      bbox=dict(facecolor='white', alpha=0.8, boxstyle='round,pad=0.5'))
               
               plt.title(f'{domain} 도메인 - ClusterSARIMA의 SARIMA 대비 성능 개선율(%) 분포', fontsize=14)
               plt.xlabel('개선율(%) (양수: ClusterSARIMA가 우수)', fontsize=12)
               plt.ylabel('빈도', fontsize=12)
               plt.legend(loc='upper right')
               plt.grid(True, alpha=0.3)
               
               # 2. 클러스터별 개선율 (클러스터 정보가 있는 경우)
               if 'cluster' in comparison_df.columns and len(comparison_df['cluster'].unique()) > 1:
                   plt.subplot(2, 1, 2)
                   
                   # 클러스터별 개선율 박스플롯
                   sns.boxplot(x='cluster', y='improvement', data=comparison_df, palette='viridis')
                   
                   # 개별 데이터 포인트 추가
                   sns.stripplot(x='cluster', y='improvement', data=comparison_df, color='black', alpha=0.3, size=3)
                   
                   # 제로 라인 추가
                   plt.axhline(y=0, color='r', linestyle='--', alpha=0.7)
                   
                   # 클러스터별 개선율 평균 표시
                   for i, cluster in enumerate(sorted(comparison_df['cluster'].unique())):
                       cluster_data = comparison_df[comparison_df['cluster'] == cluster]
                       mean_imp = cluster_data['improvement'].mean()
                       better_pct = (cluster_data['is_better'].sum() / len(cluster_data)) * 100
                       plt.text(i, mean_imp, f"{mean_imp:.1f}%\n({better_pct:.0f}% 우수)", 
                              ha='center', va='center', fontsize=9,
                              bbox=dict(facecolor='white', alpha=0.6, boxstyle='round,pad=0.2'))
                   
                   plt.title(f'{domain} 도메인 - 클러스터별 SARIMA 대비 성능 개선율', fontsize=14)
                   plt.xlabel('클러스터', fontsize=12)
                   plt.ylabel('개선율(%) (양수: ClusterSARIMA가 우수)', fontsize=12)
                   plt.grid(True, alpha=0.3)
               
               plt.tight_layout()
               plt.savefig(os.path.join(RESULTS_DIR, f'sarima_improvement_distribution_{domain}.png'), dpi=150)
               plt.show()
               
               # 개선율 스캐터플롯 (SARIMA vs ClusterSARIMA)
               plt.figure(figsize=(12, 10))
               
               plt.scatter(comparison_df['sarima_owa'], comparison_df['cluster_owa'], alpha=0.6,
                          c=comparison_df['improvement'], cmap='RdYlGn', s=50)
               
               # 기준선 (x=y) 추가
               lims = [
                   min(plt.xlim()[0], plt.ylim()[0]),
                   max(plt.xlim()[1], plt.ylim()[1])
               ]
               plt.plot(lims, lims, 'k--', alpha=0.5, label='동일 성능')
               
               # 컬러바 추가
               cbar = plt.colorbar()
               cbar.set_label('개선율 (%)', fontsize=12)
               
               plt.title(f'{domain} 도메인 - SARIMA vs ClusterSARIMA 성능 비교', fontsize=14)
               plt.xlabel('SARIMA OWA', fontsize=12)
               plt.ylabel('ClusterSARIMA OWA', fontsize=12)
               plt.grid(True, alpha=0.3)
               
               # 개선/악화 영역 표시
               plt.text(0.25, 0.8, "ClusterSARIMA 우수", transform=plt.gca().transAxes, 
                      fontsize=12, ha='center', rotation=-45, alpha=0.6)
               plt.text(0.75, 0.2, "SARIMA 우수", transform=plt.gca().transAxes, 
                      fontsize=12, ha='center', rotation=-45, alpha=0.6)
               
               plt.tight_layout()
               plt.savefig(os.path.join(RESULTS_DIR, f'sarima_vs_cluster_sarima_scatter_{domain}.png'), dpi=150)
               plt.show()
               
               # 개선율 요약 통계
               print(f"\n=== {domain} 도메인 - ClusterSARIMA의 SARIMA 대비 성능 개선율 요약 ===")
               print(f"평균 개선율: {comparison_df['improvement'].mean():.2f}%")
               print(f"중앙값 개선율: {comparison_df['improvement'].median():.2f}%")
               print(f"최대 개선율: {comparison_df['improvement'].max():.2f}%")
               print(f"최소 개선율: {comparison_df['improvement'].min():.2f}%")
               print(f"ClusterSARIMA가 우수한 시계열 비율: {(comparison_df['improvement'] > 0).mean() * 100:.2f}%")
               
               # 클러스터별 개선율 통계 (클러스터 정보가 있는 경우)
               if 'cluster' in comparison_df.columns and len(comparison_df['cluster'].unique()) > 1:
                   print(f"\n=== {domain} 도메인 - 클러스터별 개선율 통계 ===")
                   cluster_stats = comparison_df.groupby('cluster').agg({
                       'improvement': ['mean', 'median', 'min', 'max', 'count'],
                       'is_better': 'mean'  # 클러스터별 ClusterSARIMA가 우수한 비율
                   })
                   
                   # 'is_better'의 평균을 퍼센트로 변환
                   cluster_stats[('is_better', 'mean')] = cluster_stats[('is_better', 'mean')] * 100
                   
                   # 컬럼명 변경
                   cluster_stats.columns = [
                       '평균 개선율(%)', '중앙값 개선율(%)', '최소 개선율(%)', '최대 개선율(%)', 
                       '시계열 수', 'ClusterSARIMA 우수 비율(%)'
                   ]
                   
                   print(cluster_stats.round(2))

# 메인 함수
def main(log_transformed_series, exclude_heavy_models=False, include_cluster_sarima=True, optimal_params=None):
   start_time = time.time()

   # 평가할 시계열 선택
   all_ts_ids = []
   for domain in log_transformed_series:
       for ts_id in log_transformed_series[domain]:
           all_ts_ids.append((domain, ts_id))
   
   print(f"평가할 시계열: {len(all_ts_ids)}개")
   
   # 시계열 ID가 없으면 종료
   if len(all_ts_ids) == 0:
       print("오류: 평가할 시계열이 없습니다. 프로그램을 종료합니다.")
       return
   
   # 각 시계열에 대해 모델 평가
   print("모델 평가 중...")
   all_results = []

   # 전체 시계열 수 계산
   total_ts = len(all_ts_ids)
   halfway_point = total_ts // 2  # 50% 지점

   # tqdm으로 진행 상황 표시
   from tqdm.auto import tqdm

   # 시작 시간 기록
   eval_start_time = time.time()

   for i, (domain, ts_id) in enumerate(tqdm(all_ts_ids, 
                                        desc="시계열 평가", 
                                        ncols=500,
                                        leave=True)):
       try:
           # 진행 상황 출력
           if i > 0 and i % max(1, total_ts // 20) == 0:
               elapsed = time.time() - eval_start_time
               items_per_sec = i / elapsed
               remaining_items = total_ts - i
               est_remaining_sec = remaining_items / items_per_sec
               est_remaining_min = est_remaining_sec / 60
               print(f"\n진행: {i}/{total_ts} ({i/total_ts*100:.1f}%) | "
                     f"남은 예상 시간: {est_remaining_min:.1f}분")
           
           # 모델 평가 실행
           result = evaluate_time_series(
               ts_id, 
               domain,
               log_transformed_series, 
               test_size=8,  # 기본 테스트 크기
               seasonal_period=4,  # 계절성 주기가 4로 변경됨
               optimize_memory=True, 
               exclude_heavy_models=exclude_heavy_models,
               optimal_params=optimal_params,
               include_cluster_sarima=include_cluster_sarima
           )
           
           # 메모리 최적화
           result = optimize_result_memory(result)
           all_results.append(result)
           
           # 중간 결과 저장
           if i+1 == halfway_point:
               optimized_results = [optimize_result_memory(r) for r in all_results]
               
               with open(os.path.join(RESULTS_DIR, 'intermediate_results_50percent.pkl'), 'wb') as f:
                   pickle.dump(optimized_results, f)
               
               elapsed_min = (time.time() - eval_start_time) / 60
               print(f"\n50% 완료: 현재까지 {elapsed_min:.1f}분 소요")
               
               # 가비지 컬렉션
               gc.collect()
                   
       except Exception as e:
           print(f"\n시계열 {domain}/{ts_id} 처리 중 오류 발생: {e}")
           gc.collect()

   # 전체 소요 시간 계산
   total_eval_time = time.time() - eval_start_time
   print(f"\n모든 시계열 평가 완료: 총 {total_eval_time/60:.1f}분 소요")

   # 최종 결과 저장
   print("모든 시계열 평가 완료. 결과 저장 중...")
   
   optimized_results = [optimize_result_memory(r) for r in all_results]
   
   with open(os.path.join(RESULTS_DIR, 'final_results.pkl'), 'wb') as f:
       pickle.dump(optimized_results, f)
   
   # 결과 요약 및 시각화
   summary_df = summarize_model_results(all_results)
   summary_df.to_csv(os.path.join(RESULTS_DIR, 'model_summary.csv'), index=False)
   
   # 도메인별 결과 분석 및 시각화
   print("\n결과 시각화 중...")
   visualize_results(all_results, log_transformed_series)
   
   # 총 소요 시간
   total_time = time.time() - start_time
   print(f"\n총 소요 시간: {total_time:.2f}초 ({total_time/60:.2f}분)")

# 예제 실행
if __name__ == "__main__":
   # 이미 로그 변환된 시계열 데이터가 log_transformed_series 변수에 있다고 가정
   # optimal_params는 도메인별 클러스터 파라미터가 포함된 딕셔너리
   
   main(
       log_transformed_series=log_transformed_series,
       exclude_heavy_models=False, 
       include_cluster_sarima=True, 
       optimal_params=domain_results
   )